In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torchvision.transforms as transforms
from torchvision.utils import save_image

bs = 100
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./mnist_data/', train=False, transform=transforms.ToTensor(), download=False)

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=bs, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=bs, shuffle=False)


In [7]:
class VAE(nn.Module):
    def __init__(self, x_dim, h_dim1, h_dim2, z_dim):
        super(VAE, self).__init__()
        #endcoder
        self.fc1 = nn.Linear(x_dim, h_dim1)
        self.fc2 = nn.Linear(h_dim1, h_dim2)
        self.fc31 = nn.Linear(h_dim2, z_dim)
        self.fc32 = nn.Linear(h_dim2, z_dim)
        #decoder
        self.fc4 = nn.Linear(z_dim, h_dim2)
        self.fc5 = nn.Linear(h_dim2, h_dim1)
        self.fc6 = nn.Linear(h_dim1, x_dim)
    
    def encoder(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return self.fc31(h), self.fc32(h)

    def sampling(self, mu, log_var):
        std = torch.exp(0.5*log_var)
        eps = torch.rand_like(std)
        return eps.mul(std).add(mu)
        
    def decoder(self, z):
        h = F.relu(self.fc4(z))
        h = F.relu(self.fc5(h))
        return F.sigmoid(self.fc6(h))

    def forward(self, x):
        mu, log_var = self.encoder(x.view(-1, 28*28))
        z = self.sampling(mu, log_var)
        return self.decoder(z), mu, log_var


In [8]:
vae = VAE(x_dim=784, h_dim1=512, h_dim2=256, z_dim=2)
vae.cuda()

VAE(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc31): Linear(in_features=256, out_features=2, bias=True)
  (fc32): Linear(in_features=256, out_features=2, bias=True)
  (fc4): Linear(in_features=2, out_features=256, bias=True)
  (fc5): Linear(in_features=256, out_features=512, bias=True)
  (fc6): Linear(in_features=512, out_features=784, bias=True)
)

In [9]:
optimizer = optim.Adam(vae.parameters())
def loss_function(recon_x, x, mu, log_var):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')
    KLD = -0.5 * torch.sum(1 + log_var- mu.pow(2)- log_var.exp())
    return BCE + KLD

In [5]:
def train(epoch):
    vae.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.cuda()
        optimizer.zero_grad()
        
        recon_batch, mu, log_var = vae(data)
        loss = loss_function(recon_batch, data, mu, log_var)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader.dataset)))

In [11]:
def test():
    vae.eval()
    test_loss= 0
    with torch.no_grad():
        for data, _ in test_loader:
            data = data.cuda()
            recon, mu, log_var = vae(data)
            # sum up batch loss
            test_loss += loss_function(recon, data, mu, log_var).item()
        
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [12]:
for epoch in range(1, 200):
    train(epoch)
    test()
    if epoch % 20==0:
        with torch.no_grad():
            z = torch.randn(64, 2).cuda()
            sample = vae.decoder(z).cuda()
            save_image(sample.view(64, 1, 28, 28), './samples/sample_vae.'+ str(epoch) + '.png')    

Train Epoch: 1 [0/60000 (0%)]	Loss: 153.253320
Train Epoch: 1 [10000/60000 (17%)]	Loss: 152.921152
Train Epoch: 1 [20000/60000 (33%)]	Loss: 151.621475
Train Epoch: 1 [30000/60000 (50%)]	Loss: 155.386289
Train Epoch: 1 [40000/60000 (67%)]	Loss: 149.598711
Train Epoch: 1 [50000/60000 (83%)]	Loss: 154.335557
====> Epoch: 1 Average loss: 152.7962
====> Test set loss: 149.7069
Train Epoch: 2 [0/60000 (0%)]	Loss: 155.283643
Train Epoch: 2 [10000/60000 (17%)]	Loss: 143.235684
Train Epoch: 2 [20000/60000 (33%)]	Loss: 153.004395
Train Epoch: 2 [30000/60000 (50%)]	Loss: 147.735723
Train Epoch: 2 [40000/60000 (67%)]	Loss: 148.285508
Train Epoch: 2 [50000/60000 (83%)]	Loss: 145.569482
====> Epoch: 2 Average loss: 148.3827
====> Test set loss: 146.9331
Train Epoch: 3 [0/60000 (0%)]	Loss: 140.700713
Train Epoch: 3 [10000/60000 (17%)]	Loss: 147.213975
Train Epoch: 3 [20000/60000 (33%)]	Loss: 143.555430
Train Epoch: 3 [30000/60000 (50%)]	Loss: 145.707705
Train Epoch: 3 [40000/60000 (67%)]	Loss: 144.54